### A hierarchical model

Suppose ***scholastic aptitude (SA)***, denoted $u$, is distributed normally in the population with mean $\mu_0$ and variance $\sigma_0^2$:
$$
u\sim N(\mu_0, \sigma_0^2)
$$

The random variable $u$ is ***latent*** &mdash; unobservable.

A person's ***Scholastic Aptitute Test (SAT)*** score, denoted $y$, is supposed to reflect their scholastic aptitute $u$. Assume:
$$
y\mid u \sim N(u, \sigma^2)
$$
Thus, $\sigma$ is a reflection of the SAT's measurement error.

Variance in SAT score in the population has two sources: variation in SA across the population ($\sigma_0$) and variation in SAT scores among people with the same SA ($\sigma$). By *The Theory*, these two sources of variation combine additively:
$$
y\sim N(\mu_0,\sigma_0^2 + \sigma^2)
$$

### An application of Bayes' Theorem

What can we say about someone's SA if we know their test score? More precisely, what is $p(u\mid y)$?

By Bayes' Theorem,
$$
p(u\mid y) = \frac{p(y\mid u)p(u)}{p(y)}.
$$

We know $p(u)$, $p(y\mid u)$, and $p(y)$. With some algebraic perseverance, we manipulate the right hand side of the the above identify into standard Gaussian form and identify its mean and variance.

Set
$$
\tau_0 = \frac1{\sigma_0},\qquad \tau= \frac1{\sigma}.
$$
The reciprocal variances $\tau_0^2$ and $\tau^2$ are called ***precisions***.

We have:
$$
\begin{aligned}
\mathbb{E}[u\mid y] &= \frac{\tau_0^2}{\tau_0^2 + \tau^2}\mu_0 + \frac{\tau^2}{\tau_0^2 + \tau^2}y\\
\mathbb{V}[u\mid y] &= \frac{1}{\tau_0^2 + \tau^2}
\end{aligned}
$$

Let $\theta$ be the ratio of uncertainty in scholastic aptitude to uncertainty in SAT results.
$$
\theta = \frac{\sigma_0}{\sigma} = \frac{\tau}{\tau_0}.
$$
(The larger $\theta$ is, the more reliable the SAT for capturing the latent SA quantity.)

Setting
$$
t = \frac{1}{1 + \theta^2}
$$
we have
$$
\mathbb{E}[u\mid y] = t\mu_0 + (1 - t)y.
$$

The expression on the right hand side lies between $\mu_0$ and $y$.

If $\theta$ is small, $\mathbb{E}[u\mid y]$ is close to $\mu_0$. Makes sense: If $\theta$ is small then the SAT score $y$ isn't a reliable reflection of SA $u$ we hedge our bet for $u$ towards the population mean.

If $\theta$ is big, $\mathbb{E}[u\mid y]$ is close to $y$. Makes sense: If $\theta$ is big then the SAT score $y$ is a reliable reflection of SA $u$ and we don't need to hedge our bets.


$$
\begin{aligned}
u_i&\sim N(\mu_0, \sigma_0^2),&&i<m,\\
y_{ij}\mid u_i &\sim N(u_i,\sigma^2),&&j < n_i
\end{aligned}
$$

In [ ]:
import numpy as np
import pandas as pd
from collections import Counter
from matplotlib import pyplot as plt
from statsmodels.api import MixedLM
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression


mu0 = 20
sigma0 = 5
sigma = 3
m = 7

cat_dtype = pd.CategoricalDtype(categories=np.arange(m))


def make_data(n, rng=None):
    rng = np.random.default_rng(rng)
    u = rng.normal(mu0, sigma0, size=m)
    i = rng.choice(range(m), size=n)
    y = rng.normal(u[i], sigma)
    df = pd.DataFrame({"y": y, "i": i, "Intercept": 1})
    df["i"] = df["i"].astype(cat_dtype)
    return df


df_train = make_data(30)
df_test = make_data(10_000)
df_train.head()

,y,i,Intercept
0,24.411222,3,1
1,17.667072,5,1
2,20.917264,1,1
3,19.079101,1,1
4,16.644838,6,1


In [69]:
df_train["i"].value_counts()

i
1    11
6     6
3     4
5     3
0     2
2     2
4     2
Name: count, dtype: int64

In [76]:
y_train = df_train["y"]
X_train = pd.get_dummies(df_train["i"])
assert X_train.shape[1] == 7

y_test = df_test["y"]
X_test = pd.get_dummies(df_test["i"])
assert X_train.shape[1] == 7

lr_model = LinearRegression(fit_intercept=False)
lr_model.fit(X_train, y_train)
test_mse = mean_squared_error(y_test, lr_model.predict(X_test))
test_mse

61.037798213941414

In [78]:
print(lr_model.coef_)

df_train.groupby("i", observed=False)["y"].mean()

[18.0966247  19.74701551 12.4163255  24.17174297 20.0208904  17.58447808
 13.39358496]


i
0    18.096625
1    19.747016
2    12.416325
3    24.171743
4    20.020890
5    17.584478
6    13.393585
Name: y, dtype: float64

In [85]:
assert np.allclose(df_train.groupby("i", observed=False)["y"].mean(), lr_model.coef_)

In [86]:
mixed_model = MixedLM(df_train["y"], df_train[["Intercept"]], groups=df_train["i"])
fit = mixed_model.fit()
fit.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
        Mixed Linear Model Regression Results
======================================================
Model:            MixedLM Dependent Variable: y       
No. Observations: 30      Method:             REML    
No. Groups:       7       Scale:              9.3230  
Min. group size:  2       Log-Likelihood:     -80.6300
Max. group size:  11      Converged:          Yes     
Mean group size:  4.3                                 
------------------------------------------------------
            Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------
Intercept   17.975    1.544 11.641 0.000 14.949 21.002
Group Var   13.699    3.469                           
======================================================

"""

In [87]:
fit.random_effects[0]["Group Var"]

0.09057611740916587

In [88]:
df_test["i"].map(lambda i: fit.random_effects[i]["Group Var"])

0       5.295523
1       1.668555
2       5.295523
3      -4.147552
4      -4.147552
          ...   
9995   -4.147552
9996    1.668555
9997   -0.318496
9998   -4.114896
9999   -4.147552
Name: i, Length: 10000, dtype: category
Categories (7, float64): [0.090576, 1.668555, -4.147552, 5.295523, 1.526290, -0.318496, -4.114896]

In [ ]:
mixed_preds = fit.predict(df_test[["Intercept"]]) + df_test["i"].map(
    lambda i: fit.random_effects[i]["Group Var"]
).astype(float)

In [90]:
mean_squared_error(y_test, mixed_preds)

55.1060139787531